# FILE AND EXCEPTION HANDLING

### 1. Discuss the scenarios where multithreading is preferable to multiprocessing and scenarios where multiprocessing is a better choice.

**Multithreading:**
Multithreading is more efficient in cases where multiple threads need to share the same memory space and where tasks are more I/O-bound than CPU-bound.
- __I/O-bound operations:__
When a program spends most of its time waiting for input/output operations (e.g., reading from a disk, waiting for network requests), multithreading can allow other threads to perform tasks while one waits. This helps to maximize the usage of system resources.

- __Low overhead requirements:__

Since threads share the same memory space, the overhead of context switching is lower compared to processes. When tasks are lightweight, and memory sharing is crucial, multithreading is beneficial.
- __Shared data structures:__

When threads need to operate on the same data or share variables, multithreading simplifies the code by avoiding the need to serialize or transfer data between processes.
- __Limited CPU resources:__

In environments where CPU resources are constrained or where creating new processes would be costly in terms of memory usage, multithreading helps by reducing the overhead.

**Multiprocessing:**
Multiprocessing is more suitable for CPU-bound tasks where the workload can be divided across multiple CPU cores, and each process can run independently with its own memory space.

Scenarios where multiprocessing is preferable:
- __CPU-bound operations:__

If the application is performing heavy computation, multiprocessing can fully utilize the system's multiple CPU cores by distributing the workload across multiple processes.
- __Parallel execution:__

Since processes run in separate memory spaces, multiprocessing avoids the Global Interpreter Lock (GIL) in languages like Python, which can limit true parallelism in multithreaded applications. 
- __Independent tasks:__

If tasks are independent and don't need to share much data between them, multiprocessing avoids the complexity of managing shared memory and locks.
- __Fault isolation:__

Since each process runs in its own memory space, if one process crashes or misbehaves, it doesn’t affect other processes. 

### 2. Describe what a process pool is and how it helps in managing multiple processes efficiently.

A process pool is a high-level abstraction that manages a collection  of worker processes, enabling efficient handling of multiple processes. It simplifies the process of running tasks concurrently by managing the creation, execution, and termination of worker processes.

- __Reduced Overhead:__

Creating and destroying processes repeatedly can be costly due to memory allocation, CPU scheduling, and other system resources. Process pools minimize this overhead by reusing a fixed set of processes.

- __Concurrency Control:__

The pool size controls the level of concurrency. By limiting the number of processes, the system can avoid being overwhelmed by excessive context switching and resource contention.

- __Scalability:__

Process pools are scalable. By adjusting the pool size, the system can be tuned for optimal performance, ensuring efficient CPU utilization.

- __Fault Tolerance:__

If a worker process fails during execution, the pool can create a new one to take over, ensuring robustness without affecting other running processes.


### 3. Explain what multiprocessing is and why it is used in Python programs.

Multiprocessing involves running multiple independent processes, where each process can perform a task in parallel with others. Unlike multithreading, which involves multiple threads sharing the same memory space, processes in multiprocessing are completely isolated from one another and have their own memory and resources.

__Use Multiprocessing in Python:__
Python’s Global Interpreter Lock (GIL) is a mechanism that allows only one thread to execute Python bytecode at a time, even on multi-core processors. This severely limits the benefits of using multithreading in CPU-bound tasks, as the GIL becomes a bottleneck.

To overcome this limitation and achieve true parallelism, Python provides the multiprocessing module. This allows Python programs to bypass the GIL by creating separate processes, where each process runs on a different CPU core, fully utilizing multi-core processors for CPU-bound tasks.

### 4. Write a Python program using multithreading where one thread adds numbers to a list, and another thread removes numbers from the list. Implement a mechanism to avoid race conditions using threading.Lock.

In [1]:
import threading
import time
shared_list = []
list_lock = threading.Lock()
def add_to_list():
    for i in range(10):
        list_lock.acquire()
        try:
            shared_list.append(i)
            print(f"Added: {i}, List: {shared_list}")
        finally:
            list_lock.release()
        time.sleep(1)  
def remove_from_list():
    for i in range(10):
        time.sleep(1.5) 
        list_lock.acquire()
        try:
            if shared_list:
                removed = shared_list.pop(0)
                print(f"Removed: {removed}, List: {shared_list}")
            else:
                print("List is empty, nothing to remove.")
        finally:
            list_lock.release()
thread1 = threading.Thread(target=add_to_list)
thread2 = threading.Thread(target=remove_from_list)
thread1.start()
thread2.start()
thread1.join()
thread2.join()

print("Final list:", shared_list)


Added: 0, List: [0]
Added: 1, List: [0, 1]
Removed: 0, List: [1]
Added: 2, List: [1, 2]
Added: 3, List: [1, 2, 3]
Removed: 1, List: [2, 3]
Added: 4, List: [2, 3, 4]
Removed: 2, List: [3, 4]
Added: 5, List: [3, 4, 5]
Added: 6, List: [3, 4, 5, 6]
Removed: 3, List: [4, 5, 6]
Added: 7, List: [4, 5, 6, 7]
Removed: 4, List: [5, 6, 7]
Added: 8, List: [5, 6, 7, 8]
Removed: 5, List: [6, 7, 8]
Added: 9, List: [6, 7, 8, 9]
Removed: 6, List: [7, 8, 9]
Removed: 7, List: [8, 9]
Removed: 8, List: [9]
Removed: 9, List: []
Final list: []


### 5. Describe the methods and tools available in Python for safely sharing data between threads and processes.

In Python, safely sharing data between threads and processes is crucial to avoid race conditions, deadlocks, and data corruption. Both threading and multiprocessing have different mechanisms for data sharing, as threads share memory within a process while processes have isolated memory spaces. Python provides several methods and tools to handle these scenarios safely.

__1. Data Sharing Between Threads__
- threading.Lock
- threading.RLock
- threading.Semaphore
- threading.Condition
- threading.Event
- threading.Queue
  
__2. Data Sharing Between Processes__
- multiprocessing.Queue
- multiprocessing.Pipe
- multiprocessing.Manager
- multiprocessing.Value
- multiprocessing.Lock
 
__3. Tools for Synchronization__
- multiprocessing.Condition
- multiprocessing.Semaphore
- multiprocessing.Event

### 6. Discuss why it’s crucial to handle exceptions in concurrent programs and the techniques available for doing so.

Handling exceptions in concurrent programs is crucial to ensuring the stability, reliability, and robustness of a program. When multiple threads or processes run concurrently, any unhandled exception in one of them can lead to unexpected behavior, corruption of shared resources, or even the crashing of the entire program.

__Why Exception Handling Is Crucial in Concurrent Programs__

- __Prevents Silent Failures:__

In concurrent programs, exceptions may go unnoticed if not handled properly. For instance, in multithreading, if an exception occurs in a worker thread, it doesn’t directly affect the main thread, and the program might continue running without knowing something went wrong. 

- __Ensures Data Integrity:__

Concurrent programs often share data between threads or processes. If an exception is not handled, it could leave shared resources in an inconsistent or corrupt state (e.g., partially updated data, locked resources, etc.), leading to unpredictable results.

- __Avoids Resource Leaks:__

If an exception occurs during the execution of a thread or process and it is not properly handled, resources such as file handles, network connections, or memory might not be released correctly, leading to resource leaks.

- __Maintains System Stability:__

In multiprocessing, an exception in one process can cause that process to terminate prematurely. If the exception is not handled and the parent process is not informed, the overall system might not behave as expected. 

**Techniques for Handling Exceptions in Concurrent Programs:**
1. Exception Handling in Multithreading
   - Try-Except Block Inside Threads
   - Propagating Exceptions to the Main Thread
   - Using Custom Thread Classes
2. Exception Handling in Multiprocessing
   - Try-Except Block Inside Processes
   - Propagating Exceptions to the Main Process
   - Using concurrent.futures.ProcessPoolExecutor
   -  Using Manager for Shared State

### 7. Create a program that uses a thread pool to calculate the factorial of numbers from 1 to 10 concurrently. Use concurrent.futures.ThreadPoolExecutor to manage the threads

In [2]:
from concurrent.futures import ThreadPoolExecutor
import math
def factorial(n):
    print(f"Calculating factorial of {n}")
    return math.factorial(n)
numbers = range(1, 11)
with ThreadPoolExecutor() as executor:
    futures = [executor.submit(factorial, num) for num in numbers]
    for future in futures:
        try:
            result = future.result()  
            print(f"Factorial result: {result}")
        except Exception as e:
            print(f"An error occurred: {e}")


Calculating factorial of 1
Calculating factorial of 2
Calculating factorial of 3
Calculating factorial of 4
Calculating factorial of 5
Calculating factorial of 6
Calculating factorial of 7
Calculating factorial of 8
Factorial result: 1
Factorial result: 2
Factorial result: 6
Factorial result: 24
Factorial result: 120
Factorial result: 720
Factorial result: 5040
Factorial result: 40320
Calculating factorial of 9
Calculating factorial of 10
Factorial result: 362880
Factorial result: 3628800


### 8. Create a Python program that uses multiprocessing.Pool to compute the square of numbers from 1 to 10 in parallel. Measure the time taken to perform this computation using a pool of different sizes (e.g., 2, 4, 8 processes)

In [ ]:
import multiprocessing
import time
def square(n):
    return n * n
numbers = range(1, 11)
def measure_time(pool_size):
    print(f"\nUsing a pool of {pool_size} processes")
    start_time = time.time()
    with multiprocessing.Pool(pool_size) as pool:
        results = pool.map(square, numbers)
    end_time = time.time()
    print(f"Squares: {results}")
    print(f"Time taken: {end_time - start_time:.4f} seconds")

if __name__ == "__main__":
    for pool_size in [2, 4, 8]:
        measure_time(pool_size)



Using a pool of 2 processes
